In [ ]:
import torch
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from skimage import measure
import plotly.graph_objects as go
import time
import os
device = 'cuda:0'
torch.manual_seed(0)
torch.cuda.is_available()

In [ ]:
# Visualize Function
def visualize(func, thres=1e-6):
    verts, faces, _, _ = measure.marching_cubes(func, thres, spacing=(0.1, 0.1, 0.1))
    intensity = np.linalg.norm(verts, axis=1)

    fig = go.Figure(data=[go.Mesh3d(x=verts[:, 0], y=verts[:, 1], z=verts[:, 2],
                                    i=faces[:, 0], j=faces[:, 1], k=faces[:, 2],
                                    intensity=intensity,
                                    colorscale='Agsunset',
                                    opacity=0.5)])

    fig.update_layout(scene=dict(xaxis=dict(visible=False),
                                 yaxis=dict(visible=False),
                                 zaxis=dict(visible=False),
                                 bgcolor='rgb(0, 0, 0)'),
                      margin=dict(l=0, r=0, b=0, t=0))
    fig.show()

In [ ]:
# Density Functional Theory
def calculate(config, molecule, result):
    t_start = time.time()
    
    # Config
    N = config["N"]
    L = config["L"]
    dx = L / N
    Z = config["Z"]
    mix = config["mix"]
    fast = config["fast"]
    ion_ene = config["ion_ene"]
    if Z == -1:
        Z = sum(molecule["element"])
    Atom = []
    Qm = 0
    for i in range(len(molecule["element"])):
        Atom.append([molecule["element"][i], np.array([molecule["x"][i], molecule["y"][i], molecule["z"][i]]) * 1.8897])
        Qm += Atom[i][1]
    Qm /= len(Atom)
    for a in Atom:
        a[1] -= Qm

    # Kinetic Energy
    D = sp.sparse.spdiags(np.array([np.ones([N]), -2*np.ones([N]), np.ones([N])]), np.array([-1,0,1]), N, N)
    Lap = sp.sparse.kronsum(sp.sparse.kronsum(D,D), D) / (dx**2)
    T = -1/2 * Lap

    # External Energy
    Q = np.zeros((3, N, N, N))
    Q[0,:,:,:] = np.linspace(-L/2, L/2, N)[:, np.newaxis, np.newaxis]
    Q[1,:,:,:] = np.linspace(-L/2, L/2, N)[np.newaxis, :, np.newaxis]
    Q[2,:,:,:] = np.linspace(-L/2, L/2, N)[np.newaxis, np.newaxis, :]
    V_ext = 0
    for a in Atom:
        Za = a[0]
        Qa = a[1][:, np.newaxis, np.newaxis, np.newaxis]
        V_ext += -Za / (np.sqrt(np.sum((Q-Qa)*(Q-Qa), axis=0)) + 1e-6)
    V_ext = sp.sparse.diags(V_ext.reshape(N**3))
    
    # Previous Density
    if len(result["priori"]) == 0:
        ND = 0
        for a in Atom:
            Za = a[0]
            Qa = a[1][:, np.newaxis, np.newaxis, np.newaxis]
            Ra = np.sqrt(np.sum((Q-Qa)*(Q-Qa), axis=0))
            Ia = ion_ene[Za] / 27.211
            ND += (Za*np.sqrt(512*(Ia**3))) / (8*np.pi) * np.exp(-np.sqrt(8*Ia)*Ra)
        ND = ND.reshape(N**3)
        result["priori"].append(ND)
        print("Initial")
        visualize(ND.reshape(N,N,N), 1e-1)
    ND = result["priori"][-1]

    # Hartree Energy
    V_har = sp.sparse.linalg.cg(Lap, -4*np.pi*ND)
    V_har = sp.sparse.diags(V_har[0])

    # Exchange-Correlation Energy
    V_xc = -np.cbrt(3/np.pi) * np.cbrt(ND)
    V_xc = sp.sparse.diags(V_xc)
    
    # Solve
    H = (T + V_ext + V_har + V_xc).tocoo()
    H = torch.sparse_coo_tensor(indices=torch.tensor(np.vstack([H.row, H.col])), values=torch.tensor(H.data), size=H.shape).to(device)
    if fast:
        H = H.float()
    fn = [2 for i in range(Z//2)]
    if Z % 2 == 1:
        fn.append(1)
    eigval, eigvec = torch.lobpcg(H, len(fn), largest=False)

    # Density
    orbits = eigvec.T.detach().cpu().numpy()
    orbits = orbits / np.sqrt(np.sum(orbits*orbits*(dx**3), axis=1))[:, np.newaxis]
    ND = np.zeros(N**3, dtype=np.float32)
    for ne, orb in zip(fn, orbits):
        ND += ne*(orb**2)
    ND *= sum(molecule["element"]) / Z
        
    # Mix
    Dif = np.sum(np.abs(ND - result["priori"][-1]) * (dx**3))
    Pri = ND*(mix*Dif) + result["priori"][-1]*(1-mix*Dif)
    
    # Result
    result["priori"].append(Pri)
    result["density"].append(ND)
    result["difference"].append(Dif)
    
    t_end = time.time()
    print("Iteration:", len(result["density"]), " / Time:", t_end-t_start)
    print("Density difference:", Dif)
    visualize(ND.reshape(N,N,N), 1e-1)

In [ ]:
Config = {
    "N": 100,
    "L": 30,
    "Z": -1,
    "mix": 0.002,
    "fast": True,
    "ion_ene": [0, 13.59844, 24.58738, 
                5.39171, 9.32269, 8.29803, 11.26030, 14.53414, 13.61806, 17.42282, 21.5646]
}
Molecule = {
"name": "Dopamine",
"element": [
  8,
  8,
  7,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1
],
"x": [
  -2.2392,
  -3.3557,
  4.4081,
  2.1628,
  0.704,
  2.9862,
  -0.0999,
  0.1434,
  -1.4642,
  -1.2209,
  -2.0247,
  2.5111,
  2.3332,
  2.849,
  2.6457,
  0.3315,
  0.7594,
  -1.6445,
  4.5468,
  4.7362,
  -3.1541,
  -3.5639
],
"y": [
  1.9626,
  -0.5612,
  0.2624,
  -0.0212,
  -0.1603,
  0.1008,
  0.9759,
  -1.4267,
  0.8456,
  -1.557,
  -0.4208,
  -0.8817,
  0.8564,
  -0.7888,
  0.9593,
  1.9659,
  -2.3195,
  -2.5496,
  1.0868,
  -0.5285,
  1.6866,
  -1.5074
],
"z": [
  0.0548,
  0.3868,
  0.3445,
  -0.6613,
  -0.385,
  0.6289,
  -0.2919,
  -0.2187,
  -0.0326,
  0.0407,
  0.1336,
  -1.2481,
  -1.2993,
  1.2541,
  1.2192,
  -0.4187,
  -0.2869,
  0.1686,
  -0.2388,
  -0.2089,
  0.2377,
  0.4721
]
}
Result = {
    "priori": [],
    "density": [],
    "difference": []
}
Load = None
Save = None

In [ ]:
if Load is not None and os.path.exists(Load):
    Result["priori"].append(np.load(Load))

In [ ]:
for i in range(100):
    calculate(Config, Molecule, Result)
    if Result["difference"][-1] < 2:
        break

In [ ]:
if Save is not None:
    np.save(Save, Result["priori"][-2])